# 個人課題1

- Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する
    - リポジトリ名
    - 主要な言語
    - スターの数
- 上記のデータを保存するためのDBを作成し，スクレイピングしたデータを保存する
    - GithubのAPIは使わないこと（スクレイピングの課題なので）
    - ちゃんと`time.sleep(1)`入れてね
- 保存したデータをSELECT文で表示する
- ソースコードはGithubフローに従って管理すること
    1. ブランチ作成
    2. コミット
    3. プッシュ
    4. プルリク
    5. マージ
- GoogleクラスルームでリポジトリURLを提出
- 提出期限：11月25日（火）23時59分

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

# スター数の文字列を整数に変換
def convert_stars_to_int(star_str):
    if not star_str:
        return 0
    text = star_str.strip()
    try:
        if 'k' in text:
            return int(float(text.replace('k', '')) * 1000)
        return int(text.replace(',', ''))
    except ValueError:
        return 0

def main():
    db_name = 'google_repos.db'
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # テーブル初期化
    cursor.execute('DROP TABLE IF EXISTS repositories')
    cursor.execute('''
    CREATE TABLE repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        language TEXT,
        stars INTEGER
    )
    ''')
    conn.commit()

    print("スクレイピング開始")
    base_url = "https://github.com/orgs/google/repositories"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    }

    page = 1
    total_repos = 0
    # 無限ループでページを巡回
    while True:
        target_url = f"{base_url}?page={page}"
        print(f"アクセス中: Page {page} ... ", end="", flush=True)
        # HTTPリクエスト送信
        try:
            response = requests.get(target_url, headers=headers)
            if response.status_code != 200:
                print(f"\nエラー: ステータスコード {response.status_code}")
                break
            soup = BeautifulSoup(response.text, 'html.parser')
            # リポジトリ名が入っているh3タグをすべて探す
            repo_headers = soup.find_all('h3')
            # そのページにリポジトリが1つもなければ終了
            if not repo_headers:
                print("これ以上リポジトリがありません。終了します。")
                break
            # ページ内のリポジトリ情報を格納するリスト
            repos_in_page = []
            # 各リポジトリ情報を抽出
            for h3 in repo_headers:
                a_tag = h3.find('a') # リポジトリ名のリンクタグ
                if not a_tag:
                    continue 
                name = a_tag.get_text(strip=True)
                # 行全体を取得
                row_item = h3.find_parent('li') or h3.find_parent(class_='Box-row')
                if row_item:
                    # 言語
                    lang_tag = row_item.find('span', itemprop='programmingLanguage') 
                    if not lang_tag: 
                        color_dot = row_item.find('span', class_='repo-language-color')
                        if color_dot:
                            lang_tag = color_dot.find_next_sibling('span')
                    language = lang_tag.get_text(strip=True) if lang_tag else "Unknown"
                    # スター
                    star_link = row_item.find('a', href=lambda x: x and x.endswith('/stargazers'))
                    if star_link:
                        raw_stars = star_link.get_text(strip=True)
                        stars = convert_stars_to_int(raw_stars)
                    else:
                        stars = 0
                    repos_in_page.append((name, language, stars))

            # 取得した件数が0なら終了
            if not repos_in_page:
                print("有効なデータが見つかりませんでした。終了します。")
                break

            # データベースへ保存
            cursor.executemany('INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)', repos_in_page)
            conn.commit() # コミット
            # 進捗表示
            count = len(repos_in_page)
            total_repos += count
            print(f"取得完了 ({count}件) -> 累計 {total_repos}件")
        # 例外処理
        except Exception as e:
            print(f"\n予期せぬエラーが発生しました: {e}")
            break
        # 次のページへ
        page += 1
        # サーバー負荷軽減のための待機
        time.sleep(1)
    print("---") 
    print(f"スクレイピング終了。合計 {total_repos} 件保存しました！")
    # 件数カウント
    cursor.execute('SELECT count(*) FROM repositories') 
    db_count = cursor.fetchone()[0]
    print(f"DB登録件数確認: {db_count} 件") 
    print("\nスター数TOP5")
    cursor.execute('SELECT * FROM repositories ORDER BY stars DESC LIMIT 5') # スター数上位5件取得
    for row in cursor.fetchall(): # 各行を表示
        print(f"{row[1]} (Lang: {row[2]}, Stars: {row[3]})") # リポジトリ名、言語、スター数
    conn.close()
# メイン関数実行
if __name__ == "__main__":
    main()

スクレイピング開始
アクセス中: Page 1 ... 取得完了 (30件) -> 累計 30件
アクセス中: Page 2 ... 取得完了 (30件) -> 累計 60件
アクセス中: Page 3 ... 取得完了 (30件) -> 累計 90件
アクセス中: Page 4 ... 取得完了 (30件) -> 累計 120件
アクセス中: Page 5 ... 取得完了 (30件) -> 累計 150件
アクセス中: Page 6 ... 取得完了 (30件) -> 累計 180件
アクセス中: Page 7 ... 取得完了 (30件) -> 累計 210件
アクセス中: Page 8 ... 取得完了 (30件) -> 累計 240件
アクセス中: Page 9 ... 取得完了 (30件) -> 累計 270件
アクセス中: Page 10 ... 取得完了 (30件) -> 累計 300件
アクセス中: Page 11 ... 取得完了 (30件) -> 累計 330件
アクセス中: Page 12 ... 取得完了 (30件) -> 累計 360件
アクセス中: Page 13 ... 取得完了 (30件) -> 累計 390件
アクセス中: Page 14 ... 取得完了 (30件) -> 累計 420件
アクセス中: Page 15 ... 取得完了 (30件) -> 累計 450件
アクセス中: Page 16 ... 取得完了 (30件) -> 累計 480件
アクセス中: Page 17 ... 取得完了 (30件) -> 累計 510件
アクセス中: Page 18 ... 取得完了 (30件) -> 累計 540件
アクセス中: Page 19 ... 取得完了 (30件) -> 累計 570件
アクセス中: Page 20 ... 取得完了 (30件) -> 累計 600件
アクセス中: Page 21 ... 取得完了 (30件) -> 累計 630件
アクセス中: Page 22 ... 取得完了 (30件) -> 累計 660件
アクセス中: Page 23 ... 取得完了 (30件) -> 累計 690件
アクセス中: Page 24 ... 取得完了 (30件) -> 累計 720件
アクセス中: Page 25 ...